In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as ticker
from processing import *

pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)
pd.set_option('mode.chained_assignment',  None)

merged_data = pd.read_csv('./data/final_merge_bus_data.csv')

# getrouteinfoall
getrouteinfoall = pd.read_csv('./data/getRouteInfoAll.csv', encoding = 'cp949', index_col = 0)
getrouteinfoall_df = getrouteinfoall[['ROUTE_CD','ROUTE_NO']]

### gestationbyrouteall
getstationbyrouteall = pd.read_csv('./data/getStationByRouteAll.csv', encoding = 'cp949', index_col = 0)
getstationbyrouteall_df1 = getstationbyrouteall[['ROUTE_CD','BUSSTOP_SEQ','BUSSTOP_TP','GPS_LATI','GPS_LONG']]
getstationbyrouteall_df2 = getstationbyrouteall[['ROUTE_CD','BUS_STOP_ID','BUSSTOP_SEQ','BUSSTOP_TP','GPS_LATI','GPS_LONG']]

In [3]:
merged_data_copy = data_preprocessing.merged_data_preprocessing(merged_data)
merged_data_copy.head(2)

,노선,정류장순번,정류장,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,DAY
0,102,1,수통골기점지,0,0,0,0,0,0,0,0,0,7,0,0,0,0,0,0,0,6,0,1
1,102,2,수통골삼거리,1,4,17,17,24,9,17,24,32,37,50,48,43,17,24,26,10,3,0,1


---

# 히트맵 그리기

In [4]:
# 각 노선별로 평균 데이터프레임 생성하는 코드 ex) AVERAGE_DATA_102 : 102번 버스의 평균값
bus_numbers = [102, 103, 105, 106, 108, 115, 119, 201, 211, 301, 311, 314, 511, 603, 604, 605, 613, 615, 703, 704, 705, 706, 802]
average_data_dict = {}

for bus_number in bus_numbers:
    average_data = make_data.bus_average_data(merged_data_copy, bus_number)
    average_data_dict[bus_number] = average_data

for bus_number in bus_numbers:
    df_name = f"AVERAGE_DATA_{bus_number}"
    globals()[df_name] = average_data_dict[bus_number]

In [5]:
# 필요한 노선만 필터링한 뒤, int형으로 변환
bus_numbers = ['102', '103', '105', '106', '108', '115', '119', '201', '211', '301', '311', '314', '511', '603', '604', '605', '613', '615', '703', '704', '705', '706', '802']
ROUTE_BUS_NUMBER = getrouteinfoall_df[getrouteinfoall_df['ROUTE_NO'].isin(bus_numbers)]
ROUTE_BUS_NUMBER['ROUTE_NO'] = ROUTE_BUS_NUMBER['ROUTE_NO'].astype(int)

In [6]:
bus_numbers = [102, 103, 105, 106, 108, 115, 119, 201, 211, 301, 311, 314, 511, 603, 604, 605, 613, 615, 703, 704, 705, 706, 802] # 위에서 str타입으로 불러왔으니 다시 불러오기

for bus_number in bus_numbers: # 노선(버스)별로 BUSSTOP_TP를 필터링해줌
    variable_name = f"BUS_{bus_number}_BUSSTOP_TP" # BUS_102_BUSSTOP_TP
    globals()[variable_name] = make_data.get_bus_stop_tp(getstationbyrouteall_df1, ROUTE_BUS_NUMBER, bus_number)

In [7]:
# 각 버스별로 상/하행 데이터 생성
for bus_number in bus_numbers:
    average_data = globals().get(f'AVERAGE_DATA_{bus_number}')  # ex: AVERAGE_DATA_102
    busstop_tp_data = globals().get(f'BUS_{bus_number}_BUSSTOP_TP')  # ex: BUS_102_BUSSTOP_TP
    
    bus_number_str = str(bus_number).zfill(3)  # 버스번호를 3자리로 맞춤
    bus_name = f'BUS_{bus_number_str}'
    
    # 상/하행 데이터 생성
    up_data, down_data = make_data.final_data(average_data, busstop_tp_data)
    # 데이터 저장
    globals()[f'{bus_name}_UP'] = up_data
    globals()[f'{bus_name}_DOWN'] = down_data

In [7]:
# 버스 전체 히트맵 저장
# bus_numbers = [102, 103, 105, 106, 108, 115, 119, 201, 211, 301, 311, 314, 511, 603, 604, 605, 613, 615, 703, 704, 705, 706, 802]
# up_down = ['UP', 'DOWN']

# def plot_and_save_heatmaps():
#     for bus_number in bus_numbers:
#         for direction in up_down:
#             plot_bus_heatmap(bus_number, direction)
#             print(f'{bus_number}번 {direction} 버스 히트맵 생성 및 저장 완료.')

# # 이미지 파일로 저장
# plot_and_save_heatmaps()

## folium heatmapwithtime

### 출퇴근 시간대 노선
> 105번, 604번, 605번, 703번, 705번

In [8]:
# import pandas as pd
# import folium
# from folium import plugins
# from sklearn.preprocessing import MinMaxScaler

# minMaxScaler = MinMaxScaler()

# BUS_NUM_6_TO_22 = [102, 103, 105, 106, 108, 115, 119, 201, 211, 301, 311, 314, 511, 603, 604, 605, 613, 615, 703, 704, 705, 706, 802]
# UP_DOWN_6_TO_22 = ['UP', 'DOWN']
# time_series = ['6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22']

# for bus_num in BUS_NUM_6_TO_22:
#     for up_down in UP_DOWN_6_TO_22:
#         # min-max 스케일링
#         bus_num_str = str(bus_num).zfill(3) # 3자리 숫자로 맞춰줌
#         data_to_scale = eval(f'BUS_{bus_num_str}_{up_down}.iloc[:,:19]')  # Extract the data for scaling
#         scaled_data = minMaxScaler.fit_transform(data_to_scale)  # Fit and transform the scaler
#         BUS_minMaxScaled = pd.DataFrame(scaled_data, index=data_to_scale.index, columns=data_to_scale.columns)
#         BUS_concat = pd.concat([BUS_minMaxScaled, eval(f'BUS_{bus_num_str}_{up_down}.iloc[:, 19:]')], axis=1)
        
#         # 히트맵 데이터 생성
#         heat_data = [[[row['GPS_LATI'], row['GPS_LONG'], row[time]] for index, row in BUS_concat.iterrows()] for time in time_series]
        
#         # 히트맵 folium 지도 생성 및 저장
#         m = folium.Map(location=[36.3504119, 127.3845475], tiles='cartodbdark_matter', zoom_start=13)
#         heat_map = plugins.HeatMapWithTime(heat_data, index=[time for time in time_series], auto_play=False, radius=20, max_opacity=1, gradient={0.1: 'blue', 0.3: 'lime', 0.6: 'yellow', 1: 'red'})
#         heat_map.add_to(m)
#         # m.save(f'./HEATMAP_HTML_FILE/BUS_{bus_num_str}_{up_down}.html')
#         # await map_to_png(f'./HEATMAP_CAPTURE/BUS_{bus_num_str}_{up_down}', m) # 출처 : https://yeomss.tistory.com/276


---

BUS_SPEED_DATA

In [8]:
import pandas as pd

merged_df = pd.read_csv('./merged_df.csv', encoding = 'cp949')
merged_df = data_preprocessing.merged_df_preprocessing(merged_df)
merged_df.head(2)

,노선,정류장구간,5:00,6:00,7:00,8:00,9:00,10:00,11:00,12:00,13:00,14:00,15:00,16:00,17:00,18:00,19:00,20:00,21:00,22:00,23:00,평균값
0,102,수통골삼거리-41670▶한밭대학교-41690,24.0,28.0,17.5,16.0,18.8,19.5,19.3,18.8,18.8,21.3,21.8,19.5,18.0,17.3,15.0,21.0,23.0,20.3,17.0,19.7
1,102,한밭대학교-41690▶삼성화재연수원-41730,25.3,30.8,31.0,27.5,27.8,26.5,21.3,21.5,23.8,21.3,21.5,19.8,20.3,19.8,25.5,22.5,23.0,20.5,21.0,23.7


In [9]:
# 버스별 데이터프레임 생성 코드

times = list(range(6, 23))
bus_numbers = [102, 103, 105, 106, 108, 115, 119, 201, 211, 301, 311, 314, 511, 603, 604, 605, 613, 615, 703, 704, 705, 706, 802] # 위에서 str타입으로 불러왔으니 다시 불러오기

for bus_number in bus_numbers:
    variable_name = f"BUS_SPEED_{bus_number}_MEAN" # BUS_SPEED_102_MEAN
    globals()[variable_name] = make_data.bus_speed(merged_df, bus_number)
    
for bus_number in bus_numbers:
    variable_name = f"BUS_SPEED_ROUTE_{bus_number}" # BUS_SPEED_ROUTE_102
    globals()[variable_name] = make_data.get_bus_stop_tp(getstationbyrouteall_df2, ROUTE_BUS_NUMBER, bus_number)
    
for bus_number in bus_numbers:
    variable_name = f'BUS_SPEED_{bus_number}_MEAN_COORDINATE' # BUS_SPEED_102_MEAN_COORDINATE
    mean_variable = globals()[f'BUS_SPEED_{bus_number}_MEAN'] 
    route_variable = globals()[f'BUS_SPEED_ROUTE_{bus_number}']
    globals()[variable_name] = make_data.make_coordinate(mean_variable, route_variable)
    
for bus_number in bus_numbers:
    for time in times:
        variable_name = f'BUS_SPEED_{bus_number}_MEAN_COORDINATE'
        coordinate_data = globals()[variable_name]
        final_data_variable_name = f'BUS_SPEED_{bus_number}_MEAN_COORDINATE_{time}' # BUS_SPEED_102_MEAN_COORDINATE_7
        globals()[final_data_variable_name] = make_data.speed_final_data(coordinate_data, time)
        
for bus_number in bus_numbers:
    for time in times:
        variable_name = f'BUS_SPEED_{bus_number}_MEAN_COORDINATE_{time}'
        up_variable_name = f'BUS_SPEED_{bus_number}_MEAN_COORDINATE_{time}_UP'
        down_variable_name = f'BUS_SPEED_{bus_number}_MEAN_COORDINATE_{time}_DOWN' # BUS_SPEED_102_MEAN_COORDINATE_UP
        
        coordinate_data = globals()[variable_name]
        up_data, down_data = make_data.make_up_down(coordinate_data)
        
        globals()[up_variable_name] = up_data
        globals()[down_variable_name] = down_data

In [19]:
# import os

# locations = ['UP','DOWN']

# # 버스 별 속도 map 이미지 저장 폴더 만들고 UP/DOWN 구분하는 폴더 만들기
# for bus_number in bus_numbers:
#     if not os.path.exists(f"./SPEED_VISUALIZE_IMAGE/BUS_{bus_number}"):
#         os.makedirs(f"./SPEED_VISUALIZE_IMAGE/BUS_{bus_number}")
#     for location in locations:
#         if not os.path.exists(f"./SPEED_VISUALIZE_IMAGE/BUS_{bus_number}/BUS_{bus_number}_{location}"):
#             os.makedirs(f"./SPEED_VISUALIZE_IMAGE/BUS_{bus_number}/BUS_{bus_number}_{location}")

In [21]:
# import os

# locations = ['UP','DOWN']

# # 버스 별 속도 map HTML 저장 폴더 만들고 UP/DOWN 구분하는 폴더 만들기
# for bus_number in bus_numbers:
#     if not os.path.exists(f"./SPEED_HTML/BUS_{bus_number}"):
#         os.makedirs(f"./SPEED_HTML/BUS_{bus_number}")
#     for location in locations:
#         if not os.path.exists(f"./SPEED_HTML/BUS_{bus_number}/BUS_{bus_number}_{location}"):
#             os.makedirs(f"./SPEED_HTML/BUS_{bus_number}/BUS_{bus_number}_{location}")

---

In [10]:
# 만든 지도를 png 파일로 캡쳐해서 저장하는 함수
async def map_to_png(target, bus_num, location, m): # target은 저장할 파일 이름
    html = m.get_root().render()
    browser = await launch(headless=True)

    page = await browser.newPage()
    with utilities.temp_html_filepath(html) as fname:
        await page.goto('file://{path}'.format(path=fname))

    img_data = await page.screenshot({'path': f'C:/Users/user/Desktop/final_html/speed/SPEED_VISUALIZE_IMAGE/BUS_{bus_num}/BUS_{bus_num}_{location}/{target}.png', 'fullPage': 'true', })
    await browser.close()

In [10]:
import folium
from shapely.wkt import loads
import pandas as pd
from folium import utilities
from pyppeteer import launch

# Define the times
times = list(range(6, 23))
bus_numbers = [102, 103, 105, 106, 108, 115, 119, 201, 211, 301, 311, 314, 511, 603, 604, 605, 613, 615, 703, 704, 705, 706, 802]
locations = ['UP', 'DOWN']

# Speed range colormap
colormap = {
    'slow': 'red',
    'medium': 'green',
    'fast': 'blue'
}

async def map_to_png(target, bus_num, location, m):
    html = m.get_root().render()
    browser = await launch(headless=True)

    page = await browser.newPage()
    with utilities.temp_html_filepath(html) as fname:
        await page.goto('file://{path}'.format(path=fname))

    img_data = await page.screenshot({'path': f'./SPEED_VISUALIZE_IMAGE/BUS_{bus_num}/BUS_{bus_num}_{location}/{target}.png', 'fullPage': 'true', })
    await browser.close()

for bus_number in bus_numbers:
    for time in times:
        for location in locations:
            m = folium.Map(location=[36.3504119, 127.3845475], zoom_start=11)
            legend_text = f'{time}시'

            # LINESTRING 지도에 표시
            for idx, row in globals()[f'BUS_SPEED_{bus_number}_MEAN_COORDINATE_{time}_{location}'].iterrows():
                if row['LineString'] is not None:
                    line = loads(row['LineString'])
                    coords = line.coords
                    color = colormap.get(row['speed_range'], 'gray')
                    popup_content = f"Speed: {row[f'{time}시']} km/h"
                    folium.PolyLine(locations=coords, color=color, weight=7, popup=popup_content).add_to(m)
            
            # 범례 추가 + 시간 추가
            legend_html = f"""
            <div style="position: fixed; bottom: 50px; left: 50px; width: 100px; height: 100px; 
                border:2px solid grey; z-index:9999; font-size:14px;
                background-color:black; opacity: 0.8;">
                <p style="margin: 5px;"><span style="color: red;">&#9679;</span> <span style="color: white;">Slow</span></p>
                <p style="margin: 5px;"><span style="color: green;">&#9679;</span> <span style="color: white;">Medium</span></p>
                <p style="margin: 5px;"><span style="color: blue;">&#9679;</span> <span style="color: white;">Fast</span></p>
                <p style="margin: 5px; color: yellow; font-size: 16px;">{legend_text}</p>
            </div>
            """
            m.get_root().html.add_child(folium.Element(legend_html))
            
            # HTML파일로 저장하는 코드
            html_filename = f'./SPEED_HTML/BUS_{bus_number}/BUS_{bus_number}_{location}/BUS_SPEED_{bus_number}_MEAN_COORDINATE_{time}_{location}.html'
            m.save(html_filename)
            
            # HTML파일을 PNG파일로 변환하여 저장하는 코드
            await map_to_png(f'BUS_SPEED_{bus_number}_MEAN_COORDINATE_{time}_{location}', bus_number, location, m)


In [97]:
# import folium
# from shapely.wkt import loads
# import pandas as pd

# m = folium.Map(location=[36.365, 127.352], zoom_start=12)

# # speed range 색상 지정해주기
# colormap = {
#     'slow': 'red',
#     'medium': 'green',
#     'fast': 'blue'
# }

# # Linestring 지도에 표시하는 코드
# for idx, row in BUS_SPEED_102_MEAN_COORDINATE_17_DOWN.iterrows():
#     if row['LineString'] is not None:
#         line = loads(row['LineString'])
#         coords = line.coords
#         color = colormap.get(row['speed_range'], 'gray') # speed_range에 따른 색상 설정, 미지정 시 회색 사용
#         folium.PolyLine(locations=coords, color=color, weight=7, popup=f"Speed: {row['17시']} km/h").add_to(m) # PolyLine 객체를 생성하여 지도에 추가하고 팝업으로 속도 정보를 표시
        
# # 범례 (legend) 추가해주는 코드
# legend_html = """
# <div style="position: fixed; bottom: 50px; left: 50px; width: 100px; height: 100px; 
#      border:2px solid grey; z-index:9999; font-size:14px;
#      background-color:white; opacity: 0.8;">
#      <p style="margin: 5px;"><span style="color: red;">&#9679;</span> Slow</p>
#      <p style="margin: 5px;"><span style="color: green;">&#9679;</span> Medium</p>
#      <p style="margin: 5px;"><span style="color: blue;">&#9679;</span> Fast</p>
# </div>
# """
# m.get_root().html.add_child(folium.Element(legend_html))
# m.save('./SPEED_HTML/BUS_102/BUS_102_DOWN/BUS_SPEED_102_MEAN_COORDINATE_17_DOWN.html')
# await map_to_png('BUS_SPEED_102_MEAN_COORDINATE_17_DOWN', m) # 출처 : https://yeomss.tistory.com/276

In [16]:
# [704, 705, 802] 이 노선만
# import folium
# from shapely.wkt import loads
# import pandas as pd
# from folium import utilities
# from pyppeteer import launch

# # Define the times
# times = list(range(6, 23))
# bus_numbers = [704, 705, 802]
# locations = ['UP', 'DOWN']

# # Speed range colormap
# colormap = {
#     'slow': 'red',
#     'medium': 'green',
#     'fast': 'blue'
# }

# async def map_to_png(target, bus_num, location, m):
#     html = m.get_root().render()
#     browser = await launch(headless=True)

#     page = await browser.newPage()
#     with utilities.temp_html_filepath(html) as fname:
#         await page.goto('file://{path}'.format(path=fname))

#     img_data = await page.screenshot({'path': f'C:/Users/user/Desktop/final_html/speed/SPEED_VISUALIZE_IMAGE/BUS_{bus_num}/BUS_{bus_num}_{location}/{target}.png', 'fullPage': 'true', })
#     await browser.close()

# for bus_number in bus_numbers:
#     for time in times:
#         for location in locations:
#             m = folium.Map(location=[36.3504119, 127.3845475], zoom_start=11)
#             legend_text = f'{time}시'

#             # LINESTRING 지도에 표시
#             for idx, row in globals()[f'BUS_SPEED_{bus_number}_MEAN_COORDINATE_{time}_{location}'].iterrows():
#                 if row['LineString'] is not None:
#                     line = loads(row['LineString'])
#                     coords = line.coords
#                     color = colormap.get(row['speed_range'], 'gray')
#                     popup_content = f"Speed: {row[f'{time}시']} km/h"
#                     folium.PolyLine(locations=coords, color=color, weight=7, popup=popup_content).add_to(m)
            
#             # 범례 추가 + 시간 추가
#             legend_html = f"""
#             <div style="position: fixed; bottom: 50px; left: 50px; width: 100px; height: 100px; 
#                 border:2px solid grey; z-index:9999; font-size:14px;
#                 background-color:black; opacity: 0.8;">
#                 <p style="margin: 5px;"><span style="color: red;">&#9679;</span> <span style="color: white;">Slow</span></p>
#                 <p style="margin: 5px;"><span style="color: green;">&#9679;</span> <span style="color: white;">Medium</span></p>
#                 <p style="margin: 5px;"><span style="color: blue;">&#9679;</span> <span style="color: white;">Fast</span></p>
#                 <p style="margin: 5px; color: yellow; font-size: 16px;">{legend_text}</p>
#             </div>
#             """
#             m.get_root().html.add_child(folium.Element(legend_html))
            
#             # HTML파일로 저장하는 코드
#             html_filename = f'C:/Users/user/Desktop/final_html/speed/BUS_{bus_number}/BUS_{bus_number}_{location}/BUS_SPEED_{bus_number}_MEAN_COORDINATE_{time}_{location}.html'
#             m.save(html_filename)
            
#             # HTML파일을 PNG파일로 변환하여 저장하는 코드
#             await map_to_png(f'BUS_SPEED_{bus_number}_MEAN_COORDINATE_{time}_{location}', bus_number, location, m)


In [98]:
# await map_to_png('BUS_SPEED_102_MEAN_COORDINATE_19_DOWN', m) # 출처 : https://yeomss.tistory.com/276

In [63]:
# m.save('bus_102_10.html')

In [15]:
# from folium import utilities
# from pyppeteer import launch

# # 만든 지도를 png 파일로 캡쳐해서 저장하는 함수
# async def map_to_png(target, bus_num, location, m): # target은 저장할 파일 이름
#     html = m.get_root().render()
#     browser = await launch(headless=True)

#     page = await browser.newPage()
#     with utilities.temp_html_filepath(html) as fname:
#         await page.goto('file://{path}'.format(path=fname))

#     img_data = await page.screenshot({'path': f'./SPEED_VISUALIZE_IMAGE/BUS_{bus_num}/BUS_{bus_num}_{location}/{target}.png', 'fullPage': 'true', })
#     await browser.close()

In [114]:
# await map_to_png('bus_102_10', m) # 출처 : https://yeomss.tistory.com/276

---

In [11]:
# from PIL import Image

# # 경로 설정
# input_path = './SPEED_VISUALIZE_IMAGE/BUS_102/BUS_102_DOWN/'
# output_path = './SPEED_VISUALIZE_IMAGE/BUS_102/BUS_102_DOWN/BUS_102_DOWN.gif'

# # 이미지 파일들의 이름을 시간대 순서대로 정렬
# image_files = [f for f in os.listdir(input_path) if f.endswith('.png')]
# image_files.sort(key=lambda x: int(x.split('_')[-2]))  # 예: BUS_SPEED_102_MEAN_COORDINATE_19_DOWN.png

# # 이미지들을 순서대로 읽어 GIF 파일로 저장
# images = []
# for image_file in image_files:
#     image_path = os.path.join(input_path, image_file)
#     img = Image.open(image_path)
#     images.append(img)

# # GIF 파일로 저장
# images[0].save(output_path, save_all=True, append_images=images[1:], loop=0, duration=1000)  # duration은 이미지 간의 간격(ms)


In [12]:
from PIL import Image
import os

# 버스 번호와 위치 정의
bus_numbers = [102, 103, 105, 106, 108, 115, 119, 201, 211, 301, 311, 314, 511, 603, 604, 605, 613, 615, 703, 704, 705, 706, 802]
locations = ['UP', 'DOWN']

# 버스 번호와 위치를 순환하면서 실행
for bus_number in bus_numbers:
    for location in locations:
        # 입력 및 출력 경로 설정
        input_path = f'./SPEED_VISUALIZE_IMAGE/BUS_{bus_number}/BUS_{bus_number}_{location}/'
        output_path = f'./SPEED_VISUALIZE_IMAGE/BUS_{bus_number}/BUS_{bus_number}_{location}/BUS_{bus_number}_{location}.gif'
        
        # 이미지 파일들을 시간 순서로 정렬
        image_files = [f for f in os.listdir(input_path) if f.endswith('.png')]
        image_files.sort(key=lambda x: int(x.split('_')[-2]))
        
        # 이미지 읽고 GIF 생성
        images = []
        for image_file in image_files:
            image_path = os.path.join(input_path, image_file)
            img = Image.open(image_path)
            images.append(img)

        # GIF로 저장
        images[0].save(output_path, save_all=True, append_images=images[1:], loop=0, duration=1000)

In [13]:
# import os
# import cv2

# # 경로 설정
# input_path = './SPEED_VISUALIZE_IMAGE/BUS_102/BUS_102_DOWN/'
# output_path = './SPEED_VISUALIZE_IMAGE/BUS_102/BUS_102_DOWN/BUS_102_DOWN.mp4'

# # 이미지 파일들의 이름을 시간대 순서대로 정렬
# image_files = [f for f in os.listdir(input_path) if f.endswith('.png')]
# image_files.sort(key=lambda x: int(x.split('_')[-2]))  # 예: BUS_SPEED_102_MEAN_COORDINATE_19_DOWN.png

# # 동영상 writer 초기화
# frame = cv2.imread(os.path.join(input_path, image_files[0]))
# height, width, layers = frame.shape
# fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # 동영상 코덱 설정
# out = cv2.VideoWriter(output_path, fourcc, 1.0, (width, height))  # (출력 파일명, 코덱, FPS, 해상도)

# # 이미지들을 순서대로 읽어 동영상 파일에 추가
# for image_file in image_files:
#     image_path = os.path.join(input_path, image_file)
#     frame = cv2.imread(image_path)
#     out.write(frame)

# # 동영상 파일 저장 완료 후 리소스 해제
# out.release()
# cv2.destroyAllWindows()


In [14]:
import os
import cv2

# 버스 번호와 위치 정의
bus_numbers = [102, 103, 105, 106, 108, 115, 119, 201, 211, 301, 311, 314, 511, 603, 604, 605, 613, 615, 703, 704, 705, 706, 802]
locations = ['UP', 'DOWN']

# 버스 번호와 위치를 순환하면서 실행
for bus_number in bus_numbers:
    for location in locations:
        # 입력 및 출력 경로 설정
        input_path = f'./SPEED_VISUALIZE_IMAGE/BUS_{bus_number}/BUS_{bus_number}_{location}/'
        output_path = f'./SPEED_VISUALIZE_IMAGE/BUS_{bus_number}/BUS_{bus_number}_{location}/BUS_{bus_number}_{location}.mp4'

        # 이미지 파일들을 시간 순서로 정렬
        image_files = [f for f in os.listdir(input_path) if f.endswith('.png')]
        image_files.sort(key=lambda x: int(x.split('_')[-2]))

        # 동영상 라이터 초기화
        frame = cv2.imread(os.path.join(input_path, image_files[0]))
        height, width, layers = frame.shape
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # 동영상 코덱 설정
        out = cv2.VideoWriter(output_path, fourcc, 1.0, (width, height))  # (출력 파일명, 코덱, FPS, 해상도)

        # 이미지들을 순서대로 읽어 동영상 파일에 추가
        for image_file in image_files:
            image_path = os.path.join(input_path, image_file)
            frame = cv2.imread(image_path)
            out.write(frame)

        # 동영상 파일 저장 후 리소스 해제
        out.release()
        cv2.destroyAllWindows()